In [ ]:
import asyncio
from hume import HumeStreamClient
from hume.models.config import LanguageConfig

samples = [
    "Mary had a little lamb,",
    "Its fleece was white as snow.",
    "Everywhere the child went,",
    "The little lamb was sure to go."
]

async def main():
    client = HumeStreamClient(hume_api)
    config = LanguageConfig()
    async with client.connect([config]) as socket:
        for sample in samples:
            result = await socket.send_text(sample)
            emotions = result["language"]["predictions"][0]["emotions"]
            print(emotions)

# 检查是否已存在事件循环
loop = asyncio.get_event_loop()
if loop.is_running():
    # 如果事件循环已经在运行，则使用已有的循环
    loop.create_task(main())
else:
    # 如果没有事件循环在运行，则正常使用 asyncio.run
    asyncio.run(main())


In [1]:
import asyncio
import json
from hume import HumeStreamClient
from hume.models.config import LanguageConfig

hume_api = "bGKD8LrwTrKIHS6BQFVV8p8rrImljoCTiAQik7Yp2Phk3GiV" #需要换成你的API密钥
top_n = 10  # 保留的情感数量
all_emotions= []  # 创建一个空列表来存储所有的情感数据

samples = [
    "Mary had a little lamb,",
    "Its fleece was white as snow.",
    "Everywhere the child went,",
    "The little lamb was sure to go."
]

async def main(top_n,all_emotions):
    client = HumeStreamClient(hume_api)
    config = LanguageConfig()
    async with client.connect([config]) as socket:
        for sample in samples:
            result = await socket.send_text(sample)
            emotions = result["language"]["predictions"][0]["emotions"]
            # 使用top_n变量来控制保留的情感数量
            top_emotions = sorted(emotions, key=lambda x: x['score'], reverse=True)[:top_n]
            all_emotions.append({sample: top_emotions})  # 将每个样本的top N情感数据添加到列表中
            print(top_emotions)
    
    # 将所有情感数据写入JSON文件
    with open('top_emotions.json', 'w') as f:
        json.dump(all_emotions, f, ensure_ascii=False, indent=4)  # 确保使用非ASCII字符和格式化输出

# 检查是否已存在事件循环
loop = asyncio.get_event_loop()
if loop.is_running():
    # 如果事件循环已经在运行，则使用已有的循环
    loop.create_task(main(top_n,all_emotions))  # 传入你想要的截取数量，例如10
else:
    # 如果没有事件循环在运行，则正常使用 asyncio.run
    asyncio.run(main(top_n,all_emotions))  # 同样传入截取数量

c:\Users\qnj\anaconda3\envs\emotion\Lib\site-packages\pydantic\_internal\_fields.py:160: UserWarning: Field "model_provider" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
c:\Users\qnj\anaconda3\envs\emotion\Lib\site-packages\pydantic\_internal\_fields.py:160: UserWarning: Field "model_resource" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


[{'name': 'Love', 'score': 0.1472713202238083}, {'name': 'Calmness', 'score': 0.13555721938610077}, {'name': 'Romance', 'score': 0.09956827759742737}, {'name': 'Confusion', 'score': 0.0911511704325676}, {'name': 'Desire', 'score': 0.08503230661153793}, {'name': 'Adoration', 'score': 0.07222927361726761}, {'name': 'Admiration', 'score': 0.0637924075126648}, {'name': 'Interest', 'score': 0.06195249781012535}, {'name': 'Awkwardness', 'score': 0.061385150998830795}, {'name': 'Contentment', 'score': 0.06075143441557884}]
[{'name': 'Calmness', 'score': 0.14533287286758423}, {'name': 'Confusion', 'score': 0.14072859287261963}, {'name': 'Boredom', 'score': 0.10465148836374283}, {'name': 'Interest', 'score': 0.10246434807777405}, {'name': 'Contemplation', 'score': 0.08013632148504257}, {'name': 'Realization', 'score': 0.076319120824337}, {'name': 'Aesthetic Appreciation', 'score': 0.07604563981294632}, {'name': 'Satisfaction', 'score': 0.07013728469610214}, {'name': 'Contentment', 'score': 0.06

In [2]:
# 打印所有情感数据,这些数据被组织成了一个列表由字典组成，每个字典包含了一个样本的top N情感数据
all_emotions

[{'Mary had a little lamb,': [{'name': 'Love', 'score': 0.1472713202238083},
   {'name': 'Calmness', 'score': 0.13555721938610077},
   {'name': 'Romance', 'score': 0.09956827759742737},
   {'name': 'Confusion', 'score': 0.0911511704325676},
   {'name': 'Desire', 'score': 0.08503230661153793},
   {'name': 'Adoration', 'score': 0.07222927361726761},
   {'name': 'Admiration', 'score': 0.0637924075126648},
   {'name': 'Interest', 'score': 0.06195249781012535},
   {'name': 'Awkwardness', 'score': 0.061385150998830795},
   {'name': 'Contentment', 'score': 0.06075143441557884}]},
 {'Its fleece was white as snow.': [{'name': 'Calmness',
    'score': 0.14533287286758423},
   {'name': 'Confusion', 'score': 0.14072859287261963},
   {'name': 'Boredom', 'score': 0.10465148836374283},
   {'name': 'Interest', 'score': 0.10246434807777405},
   {'name': 'Contemplation', 'score': 0.08013632148504257},
   {'name': 'Realization', 'score': 0.076319120824337},
   {'name': 'Aesthetic Appreciation', 'score': 